In [1]:
# EEG Classification BCI Task
# Computational Intelligence Course Final Project
# Armin Panjehpour - 98101288

In [2]:
# Accessing google drive

from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/CI_Project_Dataset/'  


Mounted at /content/gdrive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
from scipy.io import loadmat

In [4]:
# Load Dataset
Data = mat = loadmat('/content/gdrive/MyDrive/Colab Notebooks/CI_Project_Dataset/CI_Project_data.mat')
print(Data.keys())
print('Dataset loaded!')

dict_keys(['__header__', '__version__', '__globals__', 'TestData', 'TrainData', 'TrainLabel'])
Dataset loaded!


In [5]:
# Training and Test Dataset
Train_Data = Data['TrainData']
Test_Data = Data['TestData']

# Training Dataset Labels
Train_Label = Data['TrainLabel']

# Dataset Size
print('Train_Data: ',Train_Data.shape)
print('Test_Data: ',Test_Data.shape)
print('Train_Label: ',Train_Label.shape)


print('Train\Test Data Created!')

Train_Data:  (30, 384, 120)
Test_Data:  (30, 384, 40)
Train_Label:  (1, 120)
Train\Test Data Created!


In [6]:
# Sampling Rate

Trial_Time = 1.5 # in seconds
Fs = Train_Data.shape[1]/Trial_Time

print('Sampling Rate is: ',Fs)

Sampling Rate is:  256.0


In [ ]:
############################################ Calculate Different Features for Each Channel ############################################

In [7]:
def Var_Feature(data):
  # variance of each channel
  # dim1: channel number, dim2: time, dim3: trial number
  var = np.var(data, axis=1)
  return var

In [8]:
def amp_hist_Feature(data, n_bins, min_amp, max_amp):
  # amplitude histogram
  # dim1: channel number, dim2: time, dim3: trial number
  n_channels = data.shape[0]
  [x_selec, y_selec, z_selec] = np.asarray(np.where((np.logical_and(1 <= data, data <= 2)) == True))
  hist_vals = []
  for i in range(n_channels):
    selected_chan = np.asarray(np.where(x_selec == i))
    selected_chan_data = data[x_selec[selected_chan],y_selec[selected_chan],z_selec[selected_chan]]
    hist_selected_chan = np.histogram(selected_chan_data, n_bins)[0]
    hist_vals.append(hist_selected_chan)
  
  return np.asarray(hist_vals)

In [36]:
def AR_Coeffs(data, order):
  # autoregressive model coefficients
  n_trials = data.shape[-1]
  n_samples = data.shape[1]
  n_channels = data.shape[0]
  Y = np.zeros((n_channels, n_samples-order, n_trials))
  Y = data[:, order:n_samples, :]

  X = np.zeros((n_trials, n_channels, n_samples-order, order+1))
  X[:,:,:,0] = 1

  Coeffs = np.zeros((n_trials, n_channels, order))

  for i in range(n_trials):
    for j in range(n_channels):
      for k in range(n_samples-order):
        for z in range(order):
          if(k-z >= 0):
            X[i,j,k,order-z-1] = data[j,k-z,i]
          else:
            X[i,j,k,order-z-1] = 0

      print(X[i,j,:,:].shape,Y[j,:,i].shape)
      Coeffs[i,j,:] = np.linalg.inv(X[i,j,:,:].T @ X[i,j,:,:]) @ X[i,j,:,:].T @ Y[j,:,i]

  return Coeffs

In [37]:
print(AR_Coeffs(Train_Data, 5).shape)

(379, 6) (379,)


LinAlgError: ignored